##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Displaying text data in TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/text_summaries"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

Using the **TensorFlow Text Summary API,** you can easily log arbitrary text and view it in TensorBoard. This can be extremely helpful to sample and examine your input data, or to record execution metadata or generated text.  You can also log diagnostic data as text that can be helpful in the course of your model development.

In this tutorial, you will try out some basic use cases of the Text Summary API.

## Setup

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
import tensorflow as tf

from datetime import datetime
import json
from packaging import version
import tempfile

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.5.0-dev20210219


## Logging a single piece of text

To understand how the Text Summary API works, you're going to simply log a bit of text and see how it is presented in TensorBoard.


In [4]:
my_text = "Hello world! 😃"

In [5]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/text_basics/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  tf.summary.text("first_text", my_text, step=0)

Now, use TensorBoard to examine the text. Wait a few seconds for the UI to spin up.

In [6]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/text_simple.png?raw=1"/> -->

## Organizing multiple text streams

If you have multiple streams of text, you can keep them in separate namespaces to help organize them, just like scalars or other data.

Note that if you log text at many steps, TensorBoard will subsample the steps to display so as to make the presentation manageable.  You can control the sampling rate using the `--samples_per_plugin` flag.

In [7]:
# Sets up a second directory to not overwrite the first one.
logdir = "logs/multiple_texts/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  with tf.name_scope("name_scope_1"):
    for step in range(20):
      tf.summary.text("a_stream_of_text", f"Hello from step {step}", step=step)
      tf.summary.text("another_stream_of_text", f"This can be kept separate {step}", step=step)
  with tf.name_scope("name_scope_2"):
    tf.summary.text("just_from_step_0", "This is an important announcement from step 0", step=0)
    

In [14]:
%tensorboard --logdir logs/multiple_texts --samples_per_plugin 'text=5'

## Markdown interpretation

TensorBoard interprets text summaries as Markdown, since rich formatting can make the data you log easier to read and understand, as shown below. (If you don't want Markdown interpretation, see [this issue](https://github.com/tensorflow/tensorboard/issues/830) for workarounds to suppress interpretation.)

In [9]:
# Sets up a third timestamped log directory under "logs"
logdir = "logs/markdown/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

some_obj_worth_noting = {
  "tfds_training_data": {
      "name": "mnist",
      "split": "train",
      "shuffle_files": "True",
  },
  "keras_optimizer": {
      "name": "Adagrad",
      "learning_rate": "0.001",
      "epsilon": 1e-07,
  },
  "hardware": "Cloud TPU",
}


# TODO: Update this example when TensorBoard is released with
# https://github.com/tensorflow/tensorboard/pull/4585
# which supports fenced codeblocks in Markdown.
def pretty_json(hp):
  json_hp = json.dumps(hp, indent=2)
  return "".join("\t" + line for line in json_hp.splitlines(True))

markdown_text = """
### Markdown Text

TensorBoard supports basic markdown syntax, including:

    preformatted code

**bold text**

| and | tables |
| ---- | ---------- |
| among | others |
"""

with file_writer.as_default():
  tf.summary.text("run_params", pretty_json(some_obj_worth_noting), step=0)
  tf.summary.text("markdown_jubiliee", markdown_text, step=0)
    

In [10]:
%tensorboard --logdir logs/markdown